# Create a meaningful movie data set
We chose not to upload the datasets we use on github because they were too big ; this notebook downloads and merges them.

The two data sources are:
- the **IMDb datasets** [link](https://datasets.imdbws.com/) that have to be downloaded and put in a `data` folder. They contains many information (ratings, etc), but not the box-office, and not in a convenient format (each SQL table is in its own tsv file).
- the **Box Office Mojo stats** [link](https://www.boxofficemojo.com/year/), that are crawled and merged with the IMDb datasets in this notebook.

The initial architecture to run this notebook should be the following:
```
    .
    └── moviePreprocessing
        ├── movieDataSetBuilder.ipynb    # this notebook 
        └── data/                        # the datasets we need
            ├── name.basics.tsv.gz
            ├── title.akas.tsv.gz
            ├── title.basics.tsv.gz
            ├── title.crew.tsv.gz
            ├── title.episode.tsv.gz
            ├── title.principals.tsv.gz
            └── title.ratings.tsv.gz
```
where each files comes from the IMDb datasets, whose specification can be found at https://www.imdb.com/interfaces/.

**The goal of this notebook is to generate a dataset containing the ratings, boxoffice, director names and main actor names, for each US and UK movie, from 2015 to 2020. This dataset will be named `movie_data_2015_2020.csv` and located in the `moviePreprocessing` folder.**

In [1]:
# Initial imports
import pandas as pd
import csv

# Constants to easily update and read the code
DATA_DIR = "./data/"
OUT_DATASET = "./movie_data_2015_2020.csv"

# Auxiliary functions

# Reads the csv with the given name.
def read_csv(name, dtype={}):
    return pd.read_csv(DATA_DIR + name,
                       compression='gzip',
                       header=0,
                       sep='\t',
                       na_values='\\N',
                       dtype=dtype,
                       quoting=csv.QUOTE_NONE)

# If st is a string, applies the conv translation, otherwise returns st unchanged 
def strtranslate(st, conv):
    if type(st) == str:
        return st.translate(conv)
    return st

## Processing the IMDb data sets
The dataset is organized in an "SQL way", each table being in its own compressed tsv file.

We are going to filter the data and merge the datasets to get all the info needed about a movie in one csv file.

### Filter relevant years only

In [2]:
# Read the `title.basics.tsv.gz` table, which contains unique information about the movies:
# type, titles, start and end years, genres, duration, adult rating
# (the datasets contains information about more than only movies, hence a `type` column)
titlesbasic = read_csv('title.basics.tsv.gz')

In [3]:
# Example to see what the table looks like:

# Extract the movies whose name contains `Star Wars: Episode V`,
# display only the first 10 results
titlesbasic[titlesbasic["primaryTitle"].str.contains("Star Wars: Episode V", na=False)].head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
78976,tt0080684,movie,Star Wars: Episode V - The Empire Strikes Back,Star Wars: Episode V - The Empire Strikes Back,0,1980.0,NaN,124.0,"Action,Adventure,Fantasy"
84322,tt0086190,movie,Star Wars: Episode VI - Return of the Jedi,Star Wars: Episode VI - Return of the Jedi,0,1983.0,NaN,131.0,"Action,Adventure,Fantasy"
1318718,tt10618318,tvEpisode,Star Wars: Episode V - The Empire Strikes Back,Star Wars: Episode V - The Empire Strikes Back,0,2019.0,NaN,NaN,Comedy
1326452,tt10631820,tvEpisode,Star Wars: Episode VI - Return of the Jedi,Star Wars: Episode VI - Return of the Jedi,0,2019.0,NaN,NaN,Comedy
1486127,tt10910686,tvEpisode,Star Wars: Episode V - The Empire Strikes Back,Star Wars: Episode V - The Empire Strikes Back,0,2019.0,NaN,127.0,Talk-Show
1515781,tt10962828,tvEpisode,Everything GREAT About Star Wars: Episode VII ...,Everything GREAT About Star Wars: Episode VII ...,0,2016.0,NaN,15.0,Comedy
1535258,tt10996576,tvEpisode,Star Wars: Episode VI - Return of the Jedi,Star Wars: Episode VI - Return of the Jedi,0,2019.0,NaN,133.0,Talk-Show
1603009,tt11114174,tvEpisode,RICK REACTS ~ Star Wars: Episode VII - The For...,RICK REACTS ~ Star Wars: Episode VII - The For...,0,2019.0,NaN,70.0,Reality-TV
1685170,tt11261064,tvEpisode,Everything GREAT About Star Wars: Episode V - ...,Everything GREAT About Star Wars: Episode V - ...,0,2017.0,NaN,22.0,Comedy
1685177,tt11261078,tvEpisode,Everything GREAT About Star Wars: Episode VI -...,Everything GREAT About Star Wars: Episode VI -...,0,2017.0,NaN,24.0,Comedy


In [4]:
# Keep only actual movies
titlesbasic = titlesbasic[titlesbasic['titleType'] == "movie"]

# Keep a one year margin for the start year, so in [2014;2021]
titlesbasic = titlesbasic[(titlesbasic['startYear'] >= 2014) & (titlesbasic['startYear'] <= 2021)]

print(titlesbasic.shape)
titlesbasic.head()

(134000, 9)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
11059,tt0011216,movie,Spanish Fiesta,La fête espagnole,0,2019.0,NaN,67.0,Drama
11636,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019.0,NaN,NaN,"Action,Crime"
16657,tt0016906,movie,Frivolinas,Frivolinas,0,2014.0,NaN,80.0,"Comedy,Musical"
61124,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70.0,Drama
64098,tt0065392,movie,Bucharest Memories,Amintiri bucurestene,0,2020.0,NaN,NaN,Documentary


In [5]:
# Re-check our former example:
titlesbasic[titlesbasic["primaryTitle"].str.contains("Star Wars: Episode V", na=False)]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
5019126,tt2488496,movie,Star Wars: Episode VII - The Force Awakens,Star Wars: Episode VII - The Force Awakens,0,2015.0,NaN,138.0,"Action,Adventure,Sci-Fi"
5034390,tt2527336,movie,Star Wars: Episode VIII - The Last Jedi,Star Wars: Episode VIII - The Last Jedi,0,2017.0,NaN,152.0,"Action,Adventure,Fantasy"


We can notice that all tv episodes are gone, and even the actual `Star Wars: Episode V` movie is gone since it's too old !

The only remaining rows are `Star Wars: Episode VII` and `Star Wars: Episode VIII`.

In [6]:
# Read the `title.akas.tsv.gz` table, which contains information about the movies depending on the region:
# (the same movie can have various titles depending on the region for example)
# title, region, language, types, some more information about the title
titlesakas = read_csv('title.akas.tsv.gz', dtype={'attributes': str})
# (specify dtype to avoid a warning being displayed
# and map an unreadable character to a comma)
titlesakas.attributes = titlesakas.attributes.apply(lambda attr: strtranslate(attr, {'\x02':','}))

print(titlesakas.shape)
titlesakas.head()

(29700882, 8)


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0.0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0.0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0.0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0.0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0.0


In [7]:
# Keep only the movies in the US and UK
titlesakas = titlesakas[(titlesakas["region"] == "US") | (titlesakas["region"] == "UK")]

In [8]:
# Merge both the `basic` movie dataset and the `alternative` movie dataset
# into a unique `moviedata` dataset, to which we will gradually add more information
moviedata = pd.merge(titlesbasic, titlesakas, how="inner", left_on="tconst", right_on="titleId")

print(moviedata.shape)
moviedata.head()

(71447, 17)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70.0,Drama,tt0062336,5,The Tango of the Widower and Its Distorting Mi...,US,NaN,imdbDisplay,NaN,0.0
1,tt0065392,movie,Bucharest Memories,Amintiri bucurestene,0,2020.0,NaN,NaN,Documentary,tt0065392,3,Bucharest Memories,US,NaN,imdbDisplay,NaN,0.0
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122.0,Drama,tt0069049,3,The Other Side of the Wind,US,NaN,imdbDisplay,NaN,0.0
3,tt0100275,movie,The Wandering Soap Opera,La telenovela errante,0,2017.0,NaN,80.0,"Comedy,Drama,Fantasy",tt0100275,6,The Wandering Soap Opera,US,NaN,imdbDisplay,NaN,0.0
4,tt0112502,movie,Bigfoot,Bigfoot,0,2017.0,NaN,NaN,"Horror,Thriller",tt0112502,3,Bigfoot,US,NaN,imdbDisplay,NaN,0.0


## Box-office data from the web

In [9]:
# Scrap boxoffice for all years, and get an unique dataframe

# Scrap boxoffice for the given year
def scrap_boxoffice(year):
    df_box = pd.read_html(f"https://www.boxofficemojo.com/year/{year}/")[0]
    df_box["Year"] = year
    return df_box

# Concatenate each year's boxoffice
boxoffice = pd.concat([scrap_boxoffice(year) for year in range(2015, 2021)])
print(boxoffice.shape)
boxoffice.head()

(1200, 12)


,Rank,Release,Genre,Budget,Running Time,Gross,Theaters,Total Gross,Release Date,Distributor,Estimated,Year
0,1,Jurassic World,-,-,-,"$652,270,625",4291,"$652,270,625",Jun 12,Universal Pictures,False,2015
1,2,Star Wars: Episode VII - The Force Awakens,-,-,-,"$651,967,269",4134,"$936,662,225",Dec 18,Walt Disney Studios Motion Pictures,False,2015
2,3,Avengers: Age of Ultron,-,-,-,"$459,005,868",4276,"$459,005,868",May 1,Walt Disney Studios Motion Pictures,False,2015
3,4,Inside Out,-,-,-,"$356,461,711",4158,"$356,461,711",Jun 19,Walt Disney Studios Motion Pictures,False,2015
4,5,Furious 7,-,-,-,"$353,007,020",4022,"$353,007,020",Apr 3,Universal Pictures,False,2015


In [10]:
# Unmatched movies
boxoffice[~boxoffice["Release"].isin(moviedata["primaryTitle"])]

,Rank,Release,Genre,Budget,Running Time,Gross,Theaters,Total Gross,Release Date,Distributor,Estimated,Year
150,151,The Green Inferno,-,-,-,"$7,192,291",1543,"$7,192,291",Sep 25,BH Tilt,False,2015
170,171,Hubble 3D,-,-,-,"$4,449,681",151,"$52,522,904",Mar 19,Warner Bros.,False,2015
198,199,The Oscar Nominated Short Films 2015: Live Action,-,-,-,"$2,412,593",280,"$2,412,593",Jan 30,Shorts International,False,2015
153,154,The Lobster2016 Re-release,-,-,-,"$8,700,374",560,"$8,700,374",May 13,A24,False,2016
173,174,The Meddler2016 Re-release,-,-,-,"$4,367,218",464,"$4,267,218",Apr 22,Sony Pictures Classics,False,2016
180,181,Kabali,-,-,-,"$3,903,095",236,"$3,903,095",Jul 21,-,False,2016
100,101,Leap!2017 Re-release,-,-,-,"$21,858,070",2705,"$21,858,070",Aug 25,The Weinstein Company,False,2017
185,186,Close Encounters of the Third Kind2017 Re-release,-,-,-,"$3,100,479",901,"$3,100,479",Sep 1,Sony Pictures Entertainment (SPE),False,2017
189,190,The Oscar Nominated Short Films 2017: Live Action,-,-,-,"$2,835,355",272,"$2,835,355",Feb 10,Shorts International,False,2017
195,196,Showtime Championship Boxing: Floyd Mayweather...,-,-,-,"$2,620,183",532,"$2,620,183",Aug 26,Fathom Events,False,2017


In [11]:
# Merge boxoffice data with IMDb
moviedata = pd.merge(moviedata, boxoffice, how="outer", left_on="primaryTitle", right_on="Release")
# Remove duplicates by only keeping the first for now
moviedata = moviedata.groupby("Release").agg(lambda x: x.iloc[0])

print(moviedata.shape)
moviedata.head()

(1090, 28)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,titleId,...,Genre,Budget,Running Time,Gross,Theaters,Total Gross,Release Date,Distributor,Estimated,Year
Release,,,,,,,,,,,,,,,,,,,,,
10 Cloverfield Lane,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",tt1179933,...,-,-,-,"$72,082,998",3427,"$72,082,998",Mar 11,Paramount Pictures,False,2016.0
100% Wolf,tt8774798,movie,100% Wolf,100% Wolf,0.0,2020.0,NaN,96.0,"Adventure,Animation,Comedy",tt8774798,...,-,-,-,"$514,957",124,"$658,764",Oct 9,Viva Pictures,False,2020.0
12 Strong,tt1413492,movie,12 Strong,12 Strong,0.0,2018.0,NaN,130.0,"Action,Drama,History",tt1413492,...,-,-,-,"$45,819,713",3018,"$45,819,713",Jan 19,Warner Bros.,False,2018.0
13 Hours,tt4172430,movie,13 Hours,13 Hours,0.0,2016.0,NaN,144.0,"Action,Drama,History",tt4172430,...,-,-,-,"$52,853,219",2917,"$52,853,219",Jan 15,Paramount Pictures,False,2016.0
1917,tt8579674,movie,1917,1917,0.0,2019.0,NaN,119.0,"Action,Drama,War",tt8579674,...,-,-,-,"$157,901,466",3987,"$159,227,644",Dec 25,Universal Pictures,False,2020.0


## Back to IMDb to names associated with the movie (director and main actors names)

### Get the list of people linked to the movie

In [12]:
################################################################
#  NOT USED SINCE `title.principals.tsv.gz` ACTUALLY CONTAINS  #
# THE MAIN PEOPLE THAT WORKED ON THE MOVIE, INCLUDING THE CREW #
################################################################

# Read the `title.crew.tsv.gz` table, which contains information about the movie's crew
# movie, director(s), writers
#crew = read_csv('title.crew.tsv.gz')

#print(crew.shape)
#crew.head()

In [13]:
# Read the `title.principals.tsv.gz` table, which contains information about the main people
# that worked on the movie (director, actor, etc)
principals = read_csv('title.principals.tsv.gz')

print(principals.shape)
principals.head()

(47576376, 6)


,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0374658,cinematographer,director of photography,NaN
3,tt0000002,1,nm0721526,director,NaN,NaN
4,tt0000002,2,nm1335271,composer,NaN,NaN


In [14]:
# Merge the people that worked on the movie in the full movie dataset

# NOT USED, SEE ABOVE
# Crew of the movie
#moviedata = pd.merge(moviedata, crew, how="left", left_on="tconst", right_on="tconst")

# Main actors and crew of the movie
moviedata = pd.merge(moviedata, principals, how="left", left_on="tconst", right_on="tconst")

print(moviedata.shape)
moviedata.head()

(10207, 33)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,titleId,...,Total Gross,Release Date,Distributor,Estimated,Year,ordering_y,nconst,category,job,characters
0,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",tt1179933,...,"$72,082,998",Mar 11,Paramount Pictures,False,2016.0,10.0,nm6618222,producer,producer,NaN
1,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",tt1179933,...,"$72,082,998",Mar 11,Paramount Pictures,False,2016.0,1.0,nm0000422,actor,NaN,"[""Howard""]"
2,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",tt1179933,...,"$72,082,998",Mar 11,Paramount Pictures,False,2016.0,2.0,nm0935541,actress,NaN,"[""Michelle""]"
3,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",tt1179933,...,"$72,082,998",Mar 11,Paramount Pictures,False,2016.0,3.0,nm0302330,actor,NaN,"[""Emmett""]"
4,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",tt1179933,...,"$72,082,998",Mar 11,Paramount Pictures,False,2016.0,4.0,nm0341174,actor,NaN,"[""Driver""]"


In [15]:
##############################################################
# UNUSED BECAUSE WE JUST WANT TO EXPORT THE DIRECTOR LIST AS #
#  IS, BUT CAN BE USEFUL FOR A QUICK ANALYSIS IN THIS FILE   #
##############################################################

# Split the director list into an array
#moviedata["directors"] = moviedata["directors"].apply(lambda x:x.split(","))

# Explode directors
#moviedata = moviedata.explode('directors')

#print(moviedata.shape)
#moviedata.head()

### Retrieve names from the ids

In [16]:
# Read the `name.basic.tsv.gz` table, which maps an unique identifier to a name
# identifier, name, profession, birth, death, known for...
names = read_csv('name.basics.tsv.gz')

In [17]:
# Merge the names with the rest of the data
moviedata = pd.merge(moviedata, names, how="inner", left_on="nconst", right_on="nconst")

print(moviedata.shape)
moviedata.head()

(10149, 38)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,titleId,...,ordering_y,nconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",tt1179933,...,10.0,nm6618222,producer,producer,NaN,Lindsey Weber,NaN,NaN,producer,"tt4530422,tt2660888,tt2548396,tt1179933"
1,tt4530422,movie,Overlord,Overlord,0.0,2018.0,NaN,110.0,"Action,Horror,Sci-Fi",tt4530422,...,9.0,nm6618222,producer,producer,NaN,Lindsey Weber,NaN,NaN,producer,"tt4530422,tt2660888,tt2548396,tt1179933"
2,tt1179933,movie,10 Cloverfield Lane,10 Cloverfield Lane,0.0,2016.0,NaN,103.0,"Action,Drama,Horror",tt1179933,...,1.0,nm0000422,actor,NaN,"[""Howard""]",John Goodman,1952.0,NaN,"actor,soundtrack,producer","tt0101410,tt1179933,tt1024648,tt1907668"
3,tt2406566,movie,Atomic Blonde,Atomic Blonde,0.0,2017.0,NaN,115.0,"Action,Thriller",tt2406566,...,3.0,nm0000422,actor,NaN,"[""Emmett Kurzfeld""]",John Goodman,1952.0,NaN,"actor,soundtrack,producer","tt0101410,tt1179933,tt1024648,tt1907668"
4,tt5968394,movie,Captive State,Captive State,0.0,2019.0,NaN,109.0,"Action,Horror,Sci-Fi",tt5968394,...,1.0,nm0000422,actor,NaN,"[""William Mulligan""]",John Goodman,1952.0,NaN,"actor,soundtrack,producer","tt0101410,tt1179933,tt1024648,tt1907668"


In [18]:
# Export the final merged data into an unique file
moviedata.to_csv(OUT_DATASET)